In [1]:
import pandas as pd

# Find out what may be parallelized

The dataset has files named `votes_{k}.csv` with `k` starting at 0 and going up to 71.

In [2]:
!ls -l data/ | grep votes_ | wc -l 

72


Let us read a single file. We will use it in order to extract schema information

In [3]:
small_data = pd.read_csv("data/votes_0.csv")

In [4]:
small_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 4 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   timestamp  object
 1   id         object
 2   region     object
 3   vote       object
dtypes: object(4)
memory usage: 61.0+ MB


In [5]:
small_data.head()

,timestamp,id,region,vote
0,2020-12-10 18:00:00,71909550-37fd-11eb-a8f2-4db64863012d,sc,red
1,2020-12-10 18:00:00,71909551-37fd-11eb-a8f2-4db64863012d,vn,red
2,2020-12-10 12:00:00,71909552-37fd-11eb-a8f2-4db64863012d,kk,red
3,2020-12-10 18:00:00,71909553-37fd-11eb-a8f2-4db64863012d,po,red
4,2020-12-10 11:00:00,71909554-37fd-11eb-a8f2-4db64863012d,po,blue


In [6]:
small_data.describe()

,timestamp,id,region,vote
count,2000000,2000000,2000000,2000000
unique,12,2000000,51,3
top,2020-12-10 17:00:00,474ec15d-37fe-11eb-a90d-4db64863012d,tn,red
freq,261118,1,240406,1001585


# Reload data

After we've learned a bit, we can be smart about how we load data.   

In [7]:
small_data = pd.read_csv("data/votes_0.csv",
                         parse_dates=["timestamp"],
                         dtype={"region": "category",
                                "vote": "category"})

In [8]:
small_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 4 columns):
 #   Column     Dtype         
---  ------     -----         
 0   timestamp  datetime64[ns]
 1   id         object        
 2   region     category      
 3   vote       category      
dtypes: category(2), datetime64[ns](1), object(1)
memory usage: 34.3+ MB


# Let's count the votes

At least for this small file; working out what needs to be done on a small sample will be useful later, when we work at large.

In [9]:
small_data["result"] = 1

In [10]:
count_per_region = (
    small_data
    .groupby(["region", "vote"])
    .result.agg("count")
    .reset_index()
)

In [11]:
count_per_region.head()

,region,vote,result
0,ai,blue,1998
1,ai,red,1708
2,ai,yellow,45
3,au,blue,18492
4,au,red,22275


In [12]:
results = list()
for region, df in count_per_region.groupby("region"):
    results.append(
        {"region": region,
         "winner": df.set_index("vote").result.idxmax()}
    )
    
winner_per_region = pd.DataFrame(results)

In [13]:
winner_per_region.head()

,region,winner
0,ai,blue
1,au,red
2,bd,blue
3,cm,red
4,cx,blue


In [14]:
delegates_per_region = pd.read_csv("data/region_delegates.csv")

In [15]:
delegates_per_region.head()

,region,delegates
0,tn,55
1,sc,38
2,po,29
3,mu,29
4,pu,20


In [16]:
winner_region_delegates = pd.merge(winner_per_region, delegates_per_region, on="region")

In [17]:
winner_region_delegates.head()

,region,winner,delegates
0,ai,blue,3
1,au,red,11
2,bd,blue,7
3,cm,red,6
4,cx,blue,3


In [18]:
winner_region_delegates.groupby("winner").delegates.sum().sort_values(ascending=False)

winner
blue    297
red     241
Name: delegates, dtype: int64